# EDA

- The os module has a perfect method to list files in a directory.
- Pandas json normalize could work here but is not necessary to convert the JSON data to a dataframe.
- You may need a nested for-loop to access each sale!
- We've put a lot of time into creating the structure of this repository, and it's a good example for future projects.  In the file functions_variables.py, there is an example function that you can import and use.  If you have any variables, functions or classes that you want to make, they can be put in the functions_variables.py file and imported into a notebook.  Note that only .py files can be imported into a notebook. If you want to import everything from a .py file, you can use the following:
```python
from functions_variables import *
```
If you just import functions_variables, then each object from the file will need to be prepended with "functions_variables"\
Using this .py file will keep your notebooks very organized and make it easier to reuse code between notebooks.

In [1]:
# (this is not an exhaustive list of libraries)
import pandas as pd
import numpy as np
import os
import json
from pprint import pprint
from functions_variables import threshold_column, ohe_column, json_to_df, encode_tags


## Data Importing

In [2]:
# load one file first to see what type of data you're dealing with and what attributes it has
tmp_df = pd.read_json('../data/housing/AK_Juneau_0.json')
# tmp_df['data']['results']

In [3]:
# Create empty DataFrame for housing
df = pd.DataFrame()
# The path to the data files
dir_path = '../data/housing/'

In [4]:
# loop over all files and put them into a dataframe
# n = 0
for file_name in sorted(os.listdir(dir_path)):
    # n += 1
    if file_name.endswith('.json'):
        # print(f'File: {file_name}')
        file_path = os.path.join(dir_path, file_name)
        tmp = json_to_df(file_path)
        tmp = tmp.dropna(axis=1, how='all')
        df = pd.concat([df, tmp], ignore_index=True)
    # if n > 1: break
# df

## Data Cleaning and Wrangling

At this point, ensure that you have all sales in a dataframe.
- Take a quick look at your data (i.e. `.info()`, `.describe()`) - what do you see?
- Is each cell one value, or do some cells have lists?
- What are the data types of each column?
- Some sales may not actually include the sale price (target).  These rows should be dropped.
- There are a lot of NA/None values.  Should these be dropped or replaced with something?
    - You can drop rows or use various methods to fills NA's - use your best judgement for each column 
    - i.e. for some columns (like Garage), NA probably just means no Garage, so 0
- Drop columns that aren't needed
    - Don't keep the list price because it will be too close to the sale price. Assume we want to predict the price of houses not yet listed

In [5]:
df.shape

(8191, 50)

In [6]:
# df.info()

In [7]:
columns = df.columns.sort_values().to_list()
# columns

In [8]:
# Rearrange DataFrame columns
df = df[columns]

In [9]:
# load and concatenate data here


In [10]:
# drop or replace values as necessary
df = df.drop(
    columns=[
        'branding',
        'community_advertisers',
        'community_description_name',
        # 'description.baths',
        'description_baths_3qtr',
        'description_baths_full',
        'description_baths_half',
        # 'description.beds',
        # 'description.garage',
        # 'description.lot_sqft',
        # 'description.sold_date',
        # 'description.sold_price',
        # 'description.sqft',
        # 'description.stories',
        'description_sub_type',
        'description_type',
        # 'description.year_built',
        'flags_is_foreclosure',
        'flags_is_new_listing',
        'flags_is_price_reduced',
        'last_update_date',
        'lead_attributes_show_contact_an_agent',
        #'list_date',
        # 'list_price',
        'listing_id',
        'location_address_city',
        'location_address_coordinate_lat',
        'location_address_coordinate_lon',
        'location_address_line',
        'location_address_postal_code',
        # 'location.address.state',
        'location_address_state_code',
        'location_county_fips_code',
        'location_county_name',
        'location_street_view_url',
        'matterport',
        'other_listings_rdc',
        'permalink',
        'photos',
        # 'price_reduced_amount',
        'primary_photo_href',
        'products_brand_name',
        # 'property_id',
        'source_agents',
        'source_plan_id',
        'source_spec_id',
        'source_type',
        'status',
        # 'tags',
        'virtual_tours'
    ],
    errors='ignore'
)

In [11]:
#add 0 to all the rows that have description_garage as NaN
df['description_garage'] = df['description_garage'].fillna(0)

#check and count the different values in the column description_stories
df['description_stories'].value_counts()

#drop all the rows that have description_stories = 10.0, 8.0, 6.0
df = df[~df['description_stories'].isin([10.0, 8.0, 6.0])]

#add 0 to all the rows that have description_garage as NaN
df['price_reduced_amount'] = df['price_reduced_amount'].fillna(0)


#drop all the rows with missing description_sold_price
df = df.dropna(subset=['description_sold_price'])

In [12]:
# Extract the date part using regex
df['list_date'] = df['list_date'].str.extract(r'(\d{4}-\d{2}-\d{2})')

# Convert to datetime format
df['list_date'] = pd.to_datetime(df['list_date'], format='%Y-%m-%d', errors='coerce')

In [13]:
# Step 1: Ensure 'list_date' values are strings
df['list_date'] = df['list_date'].astype(str)

# Step 2: Extract only the date (YYYY-MM-DD)
df['list_date'] = df['list_date'].str.extract(r'(\d{4}-\d{2}-\d{2})')

# Step 3: Convert to datetime format
df['list_date'] = pd.to_datetime(df['list_date'], format='%Y-%m-%d', errors='coerce')


In [14]:
df['list_date'] = pd.to_datetime(df['list_date'], errors='coerce')
df['description_sold_date'] = pd.to_datetime(df['description_sold_date'], errors='coerce')

In [15]:
# Calculate the number of days on the market
df['days_on_market'] = (df['description_sold_date'] - df['list_date']).dt.days

# Preview the result
print(df[['list_date', 'description_sold_date', 'days_on_market']].head())

    list_date description_sold_date  days_on_market
30 2023-12-19            2024-01-16            28.0
31 2023-11-10            2024-01-16            67.0
32 2023-12-19            2024-01-16            28.0
33 2023-07-05            2024-01-16           195.0
34 2023-10-29            2024-01-12            75.0


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6693 entries, 30 to 8185
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   description_baths       6543 non-null   float64       
 1   description_beds        6200 non-null   float64       
 2   description_garage      6693 non-null   float64       
 3   description_lot_sqft    5772 non-null   float64       
 4   description_sold_date   6693 non-null   datetime64[ns]
 5   description_sold_price  6693 non-null   float64       
 6   description_sqft        6038 non-null   float64       
 7   description_stories     5076 non-null   float64       
 8   description_year_built  6018 non-null   float64       
 9   list_date               6406 non-null   datetime64[ns]
 10  list_price              6406 non-null   float64       
 11  location_address_state  6693 non-null   object        
 12  price_reduced_amount    6693 non-null   float64     

In [17]:
from functions_variables import year_sold_listed

#apply the function to the DataFrame
df = year_sold_listed(df)


In [18]:
#drop description_sold_date and list_date
df.drop(columns=['description_sold_date', 'list_date'], inplace=True)

### Dealing with Tags

Consider the fact that with tags, there are a lot of categorical variables.
- How many columns would we have if we OHE tags, city and state?
- Perhaps we can get rid of tags that have a low frequency.

In [19]:
# OHE categorical variables / tags here
# tags will have to be done manually


In [20]:
# One-Hot Encode the columns, considering low frequency removal for 'tags'
# Encode tags with top n tags
ohe_df = encode_tags(df, 10)
ohe_df.drop(['tags'], axis=1, inplace=True)
# Get dummy variables
# dummies_state = ohe_column(df['state'], 'state')
# ohe_df = pd.concat([ohe_df, dummies_state], axis=1)
# ohe_df.drop(['state'], axis=1, inplace=True)
ohe_df.shape

(30016, 25)

In [21]:
ohe_df.head()

,tag_basement,tag_central_air,tag_community_outdoor_space,tag_dishwasher,tag_fireplace,tag_forced_air,tag_garage_1_or_more,tag_garage_2_or_more,tag_single_story,tag_two_or_more_stories,...,description_sqft,description_stories,description_year_built,list_price,location_address_state,price_reduced_amount,property_id,days_on_market,year_sold,year_listed
0,False,True,False,False,False,False,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,False,False,True,False,False,False,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,False,False,False,True,False,False,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,False,False,False,False,True,False,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,False,False,False,False,False,False,False,False,True,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
ohe_df.head()

,tag_basement,tag_central_air,tag_community_outdoor_space,tag_dishwasher,tag_fireplace,tag_forced_air,tag_garage_1_or_more,tag_garage_2_or_more,tag_single_story,tag_two_or_more_stories,...,description_sqft,description_stories,description_year_built,list_price,location_address_state,price_reduced_amount,property_id,days_on_market,year_sold,year_listed
0,False,True,False,False,False,False,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,False,False,True,False,False,False,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,False,False,False,True,False,False,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,False,False,False,False,True,False,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,False,False,False,False,False,False,False,False,True,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Dealing with Cities

- Sales will vary drastically between cities and states.  Is there a way to keep information about which city it is without OHE?
- Could we label encode or ordinal encode?  Yes, but this may have undesirable effects, giving nominal data ordinal values.
- What we can do is use our training data to encode the mean sale price by city as a feature (a.k.a. Target Encoding)
    - We can do this as long as we ONLY use the training data - we're using the available data to give us a 'starting guess' of the price for each city, without needing to encode city explicitly
- If you replace cities or states with numerical values (like the mean price), make sure that the data is split so that we don't leak data into the training selection. This is a great time to train test split. Compute on the training data, and join these values to the test data
- Note that you *may* have cities in the test set that are not in the training set. You don't want these to be NA, so maybe you can fill them with the overall mean

In [23]:
# perform train test split here
# do something with state and city

In [24]:
#set option to display all rows
pd.set_option('display.max_rows', None)  # Show all rows
df['tags'].explode().value_counts()

tags
garage_1_or_more           3772
central_air                3414
community_outdoor_space    3095
dishwasher                 2965
basement                   2914
fireplace                  2738
forced_air                 2711
single_story               2653
two_or_more_stories        2598
garage_2_or_more           2497
Name: count, dtype: int64

## Extra Data - STRETCH

> This doesn't need to be part of your Minimum Viable Product (MVP). We recommend you write a functional, basic pipeline first, then circle back and join new data if you have time

> If you do this, try to write your downstream steps in a way it will still work on a dataframe with different features!

- You're not limited to just using the data provided to you. Think/ do some research about other features that might be useful to predict housing prices. 
- Can you import and join this data? Make sure you do any necessary preprocessing and make sure it is joined correctly.
- Example suggestion: could mortgage interest rates in the year of the listing affect the price? 

In [25]:
# import, join and preprocess new data here

## EDA/ Visualization

Remember all the EDA that you've been learning about?  Now is a perfect time for it!
- Look at distributions of numerical variables to see the shape of the data and detect outliers.    
    - Consider transforming very skewed variables
- Scatterplots of a numerical variable and the target go a long way to show correlations.
- A heatmap will help detect highly correlated features, and we don't want these.
    - You may have too many features to do this, in which case you can simply compute the most correlated feature-pairs and list them
- Is there any overlap in any of the features? (redundant information, like number of this or that room...)

In [26]:
# perform EDA here

## Scaling and Finishing Up

Now is a great time to scale the data and save it once it's preprocessed.
- You can save it in your data folder, but you may want to make a new `processed/` subfolder to keep it organized